In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
pd.set_option('display.max_columns', 50)

In [2]:
def process_qoutation(source):
    df=source.drop(['Закон основание','Ссылка на КС','Участники КС', 'Даты проведения', \
                     'Адрес поставки','Заказчика','ИНН заказчика', 'КПП заказчика', \
                    'Наименование победителя','ИНН победителя','КПП победителя', \
                    'Процент снижения','Ставка победителя'],axis=1)
    df['region']=source['КПП заказчика'] // 1e7
    df=df.drop(df[(df['Статус']=='Не состоялась') | (df['Статус']=='Снята с публикации')].index)
    df=df.drop('Статус',axis=1)
    df=df.drop(df[(df['Статус контракта']!="Исполнен") & (df['Статус контракта']!="Заключен")].index)
    df=df.drop(df[df.region.isna()].index)
    df['percent']=(source['Начальная стоимость']-source['Итоговоя стоимость'])/source['Начальная стоимость']*100
    df['day_of_week']=source['Дата начала'].dt.dayofweek
    df['week']=source['Дата начала'].dt.week
    df=df.drop(['Время начала','Дата окончания','Время окончания'],axis=1) #'Дата начала',
    df=df.drop(df[df['Идентификатор СТЕ'].isna()].index)
    return df

In [13]:
pr_quot=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\КС сокращенный.xlsx',index_col=0)

In [4]:
pr_quot3=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\КС с 01 по 30 января 2020.xlsx')

In [5]:
offer_data=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\х Оферты.xlsx')
offer2_data=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\1я половина новых оферт.xlsx')
offer3_data=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\Оферты_зарегистрированные_с_01_по.xlsx')
offers=offer_data.append(offer2_data,ignore_index=True)
offers=offer_data.append(offer3_data,ignore_index=True)

In [6]:
pr_quot3=process_qoutation(pr_quot3)

In [16]:
pr_quot=pr_quot.append(pr_quot3,ignore_index=True,sort=True)

In [20]:
offer_counts=offers.pivot_table(index='Идентификатор СТЕ',values='Номер оферты', aggfunc='count').reset_index() \
    .rename(columns={'Номер оферты':'offers_count'})
pr_quot=pr_quot.merge(offer_counts,on='Идентификатор СТЕ',how='left')

In [183]:
#offer_counts.to_excel('data\offers_count.xlsx')

In [166]:
pr_quot=pr_quot.drop(pr_quot[pr_quot.percent>50].index)
pr_quot=pr_quot.drop(pr_quot[pr_quot['Количество участников КС']>13].index)
X=pr_quot.drop(['Статус контракта','Реестровый номер контракта','Номер контракта','Наименование товара',\
              'Номер оферты по итогам','Идентификатор','Начальная цена за единицу', \
             'Начальная стоимость','Итоговоя стоимость',"Количество снижений", \
                "Количество участников КС", 'percent', 'Конечная цена', 'Дата начала','day_of_week'],axis=1)
y=pr_quot['Начальная цена за единицу']

In [168]:
X=pd.get_dummies(X,'Вид')#.drop('Вид_Работа',axis=1)

In [169]:
X['type']=X['Вид_Работа']*4+X['Вид_Товар']*2+X['Вид_Услуга']

In [170]:
X=X.drop(['Вид_Работа','Вид_Товар','Вид_Услуга'],axis=1)

In [171]:
X=X.fillna(0)

In [172]:
X.head()

,region,week,Идентификатор СТЕ,Количество товара,offers_count,type
0,77.0,42,1203443.0,1.0,1.0,2
1,77.0,42,1209487.0,300.0,1.0,2
2,77.0,42,1209486.0,300.0,4.0,2
3,77.0,43,1207700.0,15.0,1.0,2
4,77.0,43,1207497.0,20.0,2.0,2


In [110]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.externals import joblib

In [156]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize

In [180]:
X_learn, X_test, y_learn, y_test=train_test_split(X,y,test_size=0.3)
clf=RandomForestRegressor(n_estimators=100)
#clf=KNeighborsRegressor()
clf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [181]:
param_grid={'max_depth':range(1,50),'min_samples_split':range(2,10),'min_samples_leaf':range(1,10)}
#param_grid={'n_neighbors':range(1,50),'leaf_size':range(10,50,10)}
srch=RandomizedSearchCV(clf,param_grid,cv=5)
srch.fit(X_learn,y_learn)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=None

In [184]:
best_clf=srch.best_estimator_
print(srch.best_params_)


{'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 41}


In [185]:
prediction=best_clf.predict(X_test)

In [186]:
result=pd.DataFrame(X_test)
result['real']=y_test
result['predicted']=prediction
result['accuracy']=pd.Series(np.abs(y_test-prediction))
result['acc_perc']=result['accuracy']/result['real']*100
r2_score(y_test,prediction)

0.4544332552833784

In [187]:
result.accuracy.describe()

count     75862.000000
mean       9748.613764
std       33830.369919
min           0.000000
25%         102.930299
50%         708.829309
75%        4308.444983
max      951114.366316
Name: accuracy, dtype: float64

In [188]:
joblib.dump(best_clf, 'randforest.model')

['randforest.model']

In [190]:
result[result.acc_perc<5].head()

,region,week,Идентификатор СТЕ,Количество товара,offers_count,type,real,predicted,accuracy,acc_perc
130481,77.0,50,20961170.0,1.0,1.0,2,48000.00,50064.551330,2064.551330,4.301149
206006,77.0,37,18648310.0,20.0,0.0,2,819.00,821.305845,2.305845,0.281544
252963,77.0,3,1359857.0,850.0,10.0,1,21.00,21.574969,0.574969,2.737946
155733,50.0,52,1458488.0,534.0,3.0,2,26.00,26.615040,0.615040,2.365539
90111,77.0,43,1391697.0,1500.0,1.0,2,24.76,25.976782,1.216782,4.914307
